# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## I have chosen the local CSV files as my dataset for these reasons: 

1. The information seams quite clean and tidy, hence I can retrieve the important data I need without much work for cleaning up.
2. The dataset files are very samll which makes they can be shared easily (e.g. use for another project).
3. Owning these dataset file means that we can amend/edit them as needed (e.g. we can add new records or correct some info within these files)

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [21]:
from dotenv import load_dotenv
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [22]:
import pandas as pd 

# load the original csv file 
CSV_FILE_NAME = "character_descriptions"
CSV_FILE_EXE = ".csv"

origin_df = pd.read_csv(CSV_FILE_NAME + CSV_FILE_EXE) 
origin_df.head()


,Name,Description,Medium,Setting
0,Emily,"A young woman in her early 20s, Emily is an as...",Play,England
1,Jack,"A middle-aged man in his 40s, Jack is a succes...",Play,England
2,Alice,"A woman in her late 30s, Alice is a warm and n...",Play,England
3,Tom,"A man in his 50s, Tom is a retired soldier and...",Play,England
4,Sarah,"A woman in her mid-20s, Sarah is a free-spirit...",Play,England


In [23]:
from pandas import DataFrame 

# copy all the descriptions from the original csv file into the 'text' column
my_df = DataFrame()
my_df["text"] = origin_df["Description"]
my_df.head()


,text
0,"A young woman in her early 20s, Emily is an as..."
1,"A middle-aged man in his 40s, Jack is a succes..."
2,"A woman in her late 30s, Alice is a warm and n..."
3,"A man in his 50s, Tom is a retired soldier and..."
4,"A woman in her mid-20s, Sarah is a free-spirit..."


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [24]:
# OpenAI setups

from openai import OpenAI
import openai

openai.api_key = OPENAI_API_KEY
ai_client = OpenAI()


In [25]:

# Create embeddings record for each text row 

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

def get_embedding(text, model=EMBEDDING_MODEL_NAME):
   text = text.replace("\n", " ")
   return ai_client.embeddings.create(input = [text], model=model).data[0].embedding




## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

### Question 2